In [33]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [34]:
# test_group --- tags:(Multirun_SuggestUsersForChat_run1__TG1 OR tags:Multirun_SuggestUsersForChat_run1__TG2 OR tags:Multirun_SuggestUsersForChat_run1__TG3) AND created:[2017-04-16 TO 2018-04-19] AND first_app:(topface OR standalonetf)
# control_group --- tags:Multirun_SuggestUsersForChat_run1__CG AND created:[2017-04-16 TO 2018-04-19] AND first_app:(topface OR standalonetf)
# old_reg_group WRONG --- created:[2018-04-01 TO 2018-04-07] AND first_app:(topface OR standalonetf)
# old_reg_group WRONG --- created:[2018-04-01 TO 2018-04-07] AND first_app:(topface OR standalonetf) AND country:RU

test_group = pd.read_csv('report-1524485834.csv')
control_group = pd.read_csv('report-1524485847.csv')
# old_reg_group = pd.read_csv('report-1524485856.csv')
old_reg_group = pd.read_csv('report-1524571078.csv')
test_group['group'] = 'test_group'
control_group['group'] = 'control_group'
old_reg_group['group'] = 'old_reg_group'
df= pd.concat([test_group, control_group, old_reg_group])

In [35]:
df.head()

,user_id,sex,isReferral,isPaid,active_days_0_1day,active_days_1day_2day,active_days_2day_3day,like_sent_0_1day,like_sent_1day_2day,like_sent_2day_3day,fake_like_sent_0_1day,fake_like_sent_1day_2day,fake_like_sent_2day_3day,like_received_0_1day,like_received_1day_2day,like_received_2day_3day,fake_like_received_0_1day,fake_like_received_1day_2day,fake_like_received_2day_3day,incoming_bomb_0_1day,incoming_bomb_1day_2day,incoming_bomb_2day_3day,message_sent_0_1day,message_sent_1day_2day,message_sent_2day_3day,message_received_0_1day,message_received_1day_2day,message_received_2day_3day,mutual_0_1day,mutual_1day_2day,mutual_2day_3day,fake_mutual_0_1day,fake_mutual_1day_2day,fake_mutual_2day_3day,incoming_fake_dialog_0_1day,incoming_fake_dialog_1day_2day,incoming_fake_dialog_2day_3day,outgoing_fake_dialog_0_1day,outgoing_fake_dialog_1day_2day,outgoing_fake_dialog_2day_3day,fist_dialog_received_0_1day,fist_dialog_received_1day_2day,fist_dialog_received_2day_3day,first_dialog_started_0_1day,first_dialog_started_1day_2day,first_dialog_started_2day_3day,fist_dialog_received_fake_0_1day,fist_dialog_received_fake_1day_2day,fist_dialog_received_fake_2day_3day,first_dialog_started_fake_0_1day,first_dialog_started_fake_1day_2day,first_dialog_started_fake_2day_3day,payment_0_1day,payment_1day_2day,payment_2day_3day,group
0,117978263,1,0,0,1,0,1,6,0,0,0,0,0,2,0,1,20,40,40,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,test_group
1,118028416,1,0,1,1,0,0,7,0,0,0,0,0,0,1,0,40,80,80,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,test_group
2,118003862,0,0,0,1,0,0,2,0,0,0,0,0,2,0,0,20,40,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,test_group
3,118009783,1,0,0,0,0,0,12,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,test_group
4,117991988,0,0,0,1,0,0,0,0,0,8,34,0,14,10,3,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,test_group


In [36]:
'user_id','sex','isReferral','isPaid','active_days','like_sent','fake_like_sent','like_received','fake_like_received',
'incoming_bomb','message_sent','message_received','mutual','fake_mutual','incoming_fake_dialog','outgoing_fake_dialog',
'fist_dialog_received','first_dialog_started','fist_dialog_received_fake','first_dialog_started_fake','payment'

('fist_dialog_received',
 'first_dialog_started',
 'fist_dialog_received_fake',
 'first_dialog_started_fake',
 'payment')

In [37]:
periods = ['_0_1day','_1day_2day','_2day_3day']
sex_dict = {0: 'women', 1: 'men'}
groups = ['test_group','control_group','old_reg_group']
metrics = ['active_days','like_sent','fake_like_sent','like_sent_nonfake','like_received','fake_like_received',
           'like_received_nonfake','incoming_bomb','mutual','fake_mutual','mutual_nonfake',
           'message_sent','outgoing_fake_dialog','message_sent_nonfake',
           'message_received','incoming_fake_dialog','message_received_nonfake',
           'fist_dialog_received','fist_dialog_received_fake','fist_dialog_received_nonfake',
           'first_dialog_started','first_dialog_started_fake','first_dialog_started_nonfake','payment']

for period in periods:
    df['like_sent_nonfake'+period] = df['like_sent'+period] - df['fake_like_sent'+period]
    df['like_received_nonfake'+period] = df['like_received'+period] - df['fake_like_received'+period]
    df['mutual_nonfake'+period] = df['mutual'+period] - df['fake_mutual'+period]
    df['message_received_nonfake'+period] = df['message_received'+period] - df['incoming_fake_dialog'+period]
    df['message_sent_nonfake'+period] = df['message_sent'+period] - df['outgoing_fake_dialog'+period]
    df['fist_dialog_received_nonfake'+period] = df['fist_dialog_received'+period] - df['fist_dialog_received_fake'+period]
    df['first_dialog_started_nonfake'+period] = df['first_dialog_started'+period] - df['first_dialog_started_fake'+period]

df = df[['group','user_id','sex','isReferral','isPaid',*[metrix + period for metrix in metrics for period in periods]]]

In [38]:
out = pd.DataFrame(index =[i+'_'+j for j in ['sign','part','mean','quantile25','quantile50','quantile75'] for i in metrics] )
for group in groups:
    for sex in [0, 1]:
        for period in periods:
            d = df[(df['group'] == group) & (df['sex'] == sex) & (df['active_days' + period] == 1)].copy()
            d = d[[x for x in df.columns if period in x]]
            d = pd.concat( [d.agg(lambda x: np.sum(np.sign(abs(x)))), d.agg(lambda x: np.sum(np.sign(abs(x)))) / d.shape[0], d.mean(), d.quantile(0.25), d.quantile(0.5), d.quantile(0.75)], axis = 0)
            d.index = [i+'_'+j for j in ['sign','part','mean','quantile25','quantile50','quantile75'] for i in metrics]
            out = pd.concat([out, d], axis = 1)
out.columns = ['_'.join([group,sex,period]) for group in groups for sex in ['women','men'] for period in ['0','1','2']]
out = out[['_'.join([group,sex,period]) for sex in ['women','men'] for period in ['0','1','2'] for group in groups]]

In [39]:
out.head()

,test_group_women_0,control_group_women_0,old_reg_group_women_0,test_group_women_1,control_group_women_1,old_reg_group_women_1,test_group_women_2,control_group_women_2,old_reg_group_women_2,test_group_men_0,control_group_men_0,old_reg_group_men_0,test_group_men_1,control_group_men_1,old_reg_group_men_1,test_group_men_2,control_group_men_2,old_reg_group_men_2
active_days_sign,1092.0,366.0,2888.0,296.0,92.0,721.0,210.0,64.0,511.0,1767.0,619.0,4593.0,535.0,164.0,1292.0,395.0,133.0,954.0
like_sent_sign,374.0,121.0,972.0,96.0,26.0,216.0,56.0,19.0,118.0,1015.0,353.0,2585.0,282.0,88.0,685.0,200.0,76.0,474.0
fake_like_sent_sign,691.0,252.0,1689.0,194.0,65.0,523.0,89.0,31.0,233.0,63.0,20.0,297.0,49.0,20.0,155.0,44.0,14.0,98.0
like_sent_nonfake_sign,760.0,270.0,1913.0,218.0,72.0,561.0,115.0,37.0,281.0,1031.0,358.0,2660.0,299.0,94.0,713.0,214.0,83.0,497.0
like_received_sign,1067.0,360.0,2829.0,264.0,81.0,645.0,180.0,60.0,450.0,1532.0,554.0,4167.0,212.0,72.0,597.0,172.0,50.0,440.0


In [40]:
writer = pd.ExcelWriter('Multirun_SuggestUsersForChat_run1_dima.xlsx')
out.to_excel(writer,'group_sex_period') 
writer.save()

In [9]:
out

,test_group_women_0,control_group_women_0,old_reg_group_women_0,test_group_women_1,control_group_women_1,old_reg_group_women_1,test_group_women_2,control_group_women_2,old_reg_group_women_2,test_group_men_0,control_group_men_0,old_reg_group_men_0,test_group_men_1,control_group_men_1,old_reg_group_men_1,test_group_men_2,control_group_men_2,old_reg_group_men_2
active_days_sign,1092.000000,366.000000,5618.000000,296.000000,92.000000,1489.000000,210.000000,64.000000,1015.000000,1767.000000,619.000000,8713.000000,535.000000,164.000000,2486.000000,395.000000,133.000000,1803.000000
like_sent_sign,374.000000,121.000000,2592.000000,96.000000,26.000000,498.000000,56.000000,19.000000,258.000000,1015.000000,353.000000,4843.000000,282.000000,88.000000,1296.000000,200.000000,76.000000,866.000000
fake_like_sent_sign,691.000000,252.000000,3349.000000,194.000000,65.000000,1064.000000,89.000000,31.000000,457.000000,63.000000,20.000000,561.000000,49.000000,20.000000,288.000000,44.000000,14.000000,181.000000
like_sent_nonfake_sign,760.000000,270.000000,4105.000000,218.000000,72.000000,1160.000000,115.000000,37.000000,566.000000,1031.000000,358.000000,4957.000000,299.000000,94.000000,1347.000000,214.000000,83.000000,915.000000
like_received_sign,1067.000000,360.000000,5489.000000,264.000000,81.000000,1347.000000,180.000000,60.000000,910.000000,1532.000000,554.000000,7790.000000,212.000000,72.000000,1037.000000,172.000000,50.000000,766.000000
fake_like_received_sign,347.000000,119.000000,2462.000000,89.000000,36.000000,674.000000,70.000000,26.000000,448.000000,1517.000000,549.000000,7336.000000,403.000000,120.000000,1876.000000,284.000000,96.000000,1249.000000
like_received_nonfake_sign,1061.000000,358.000000,5476.000000,268.000000,85.000000,1378.000000,187.000000,61.000000,926.000000,1668.000000,587.000000,8282.000000,432.000000,130.000000,2025.000000,305.000000,101.000000,1375.000000
incoming_bomb_sign,664.000000,239.000000,3005.000000,145.000000,49.000000,774.000000,113.000000,36.000000,525.000000,178.000000,62.000000,895.000000,240.000000,73.000000,1023.000000,197.000000,49.000000,874.000000
mutual_sign,383.000000,126.000000,2381.000000,148.000000,47.000000,877.000000,94.000000,32.000000,524.000000,387.000000,129.000000,1840.000000,208.000000,65.000000,973.000000,141.000000,54.000000,652.000000
fake_mutual_sign,217.000000,74.000000,1255.000000,59.000000,21.000000,395.000000,25.000000,18.000000,182.000000,191.000000,68.000000,1031.000000,115.000000,40.000000,568.000000,85.000000,36.000000,346.000000


In [41]:
metrics = ['active_days','like_sent','fake_like_sent','like_received','fake_like_received','incoming_bomb',
          'mutual','fake_mutual','mutual_nonfake','message_sent','message_sent_nonfake','message_received',
          'fist_dialog_received','first_dialog_started','payment']

for metrix in metrics:
    df[metrix] = df[metrix + periods[0]] + df[metrix + periods[1]] + df[metrix + periods[2]]

out = pd.DataFrame(index =[i+'_'+j for j in ['sign','part','mean','quantile25','quantile50','quantile75'] for i in metrics] )

for group in groups:
    for sex in [0, 1]:
        d = df[(df['group'] == group) & (df['sex'] == sex)].copy()
        d = d[[x for x in df.columns if x in metrics]]
        d = pd.concat( [d.agg(lambda x: np.sum(np.sign(abs(x)))), d.agg(lambda x: np.sum(np.sign(abs(x)))) / d.shape[0], d.mean(), d.quantile(0.25), d.quantile(0.5), d.quantile(0.75)], axis = 0)
        d.index = [i+'_'+j for j in ['sign','part','mean','quantile25','quantile50','quantile75'] for i in metrics]
        out = pd.concat([out, d], axis = 1)
out.columns = ['_'.join([group,sex]) for group in groups for sex in ['women','men']]
out = out[['_'.join([group,sex]) for sex in ['women','men'] for group in groups]]

writer = pd.ExcelWriter('Multirun_SuggestUsersForChat_run1_dima.xlsx')
out.to_excel(writer,'group_sex') 
writer.save()

In [22]:
df[(df.group == 'test_group') & (df.sex == 0) & (df.mutual_0_1day > 0)].shape[0] / df[(df.group == 'test_group') & (df.sex == 0)].shape[0]

0.34930313588850176

In [23]:
df[(df.group == 'control_group') & (df.sex == 0) & (df.mutual_0_1day > 0)].shape[0] / df[(df.group == 'control_group') & (df.sex == 0)].shape[0]

0.34285714285714286

In [24]:
df[(df.group == 'old_reg_group') & (df.sex == 0) & (df.mutual_0_1day > 0)].shape[0] / df[(df.group == 'old_reg_group') & (df.sex == 0)].shape[0]

0.31241997439180536